<a href="https://colab.research.google.com/github/mohitsshetty986/Movie-Recommendation-Systems/blob/master/PlotDescriptionBasedRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Content-Based Recommender**

##**1)Plot Description Based Recommender**

##**This recommender recommends movies similar to a particular movie based on the plot descriptions of the movies.**


In [1]:
import pandas as pd

In [3]:
from google.colab import files
uploaded = files.upload()

Saving movies_metadata.csv to movies_metadata.csv


In [4]:
metadata=pd.read_csv('movies_metadata.csv', low_memory=False)

In [5]:
metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


##**We are going to use the overview column for this movie recommendation system**



In [6]:
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

**You will compute pairwise cosine similarity scores for all movies based on their plot descriptions and then recommend movies based on their similarity scores.**


1.   Here the problem is natural language processing. We need to extract some features from the overview text data before computing the simmilarity or the dissimilarity between them. Because it is not possible to compute the similarity between any two overviews in their raw forms.
2.   For this you need to compute the word vectors of each overview which carry some semantic meaning with it. For example, President and vice president will have vector representations close to each other while president and prime minister would have representation far from each other.
3.   We will use **Term Frequency-Inverse Document Frequency** (TF-IDF) vectors to compute the word vectors in each overview. This will give a matrix where each column represents a word in the overview vocabulary (all the words(removing the duplicates) which are present in the overview column atleast once), and each row represents a movie present in the metadata.
4.   The TF-IDF score is the frequency of a word occurring in an overview, down-weighted by the number of overviews in which it occurs. This is done to reduce the importance of words that frequently occur in overviews and, therefore, their significance in computing the final similarity score.
**Example of TF-IDF vectorizer:**
The TF (term frequency) of a word is the frequency of a word (i.e. number of times it appears) in a document( in our case its movie overview).
when a 100-word document contains the term "cat" 12 times, the TF for the word "cat" is,

---

      **TF=12/100 = 0.12**
The IDF (inverse document frequency) of a word is the measure of how significant that term is in the whole corpus.
Suppose there are 0.3 million documents that contain the term "cat", then the IDF (i.e. log {DF}) is given by the total number of documents for example say 10,000,000 divided by the number of documents containing the term "cat" for example say 300,000.

---

      **IDF=log(10000000/300000) = 1.52**

You can check this [link](https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76) for better understanding of TF-IDF vectorizer.  




**We are going to use scikit-learn which has a built-in TfIdfVectorizer class to produce the TF-IDF matrix.**

We are going to remove all english words such as "the","and",etc which are unnecessary words here.
We are also going to change the NAN entries in the overview column with empty string. 


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

metadata['overview'] = metadata['overview'].fillna('')                #converting nan values with empty string

tfidf_matrix = tfidf.fit_transform(metadata['overview'])

tfidf_matrix.shape

(45466, 75827)

We got the TF-IDF matrix and from above result we find that there are total 45466 movies in the dataset we are working with and the overview column contains a sum of 75827 unique words excluding the english stop-words.

In [8]:
tfidf.get_feature_names()[9000:9010]   # words from columns 9000 to 9009 of the TF-IDF matrix

['bribed',
 'bribery',
 'bribes',
 'bribie',
 'bribing',
 'brice',
 'brick',
 'bricked',
 'bricklayer',
 'brickley']

##**We are going to use cosine similarity to calculate a numeric quantity that denotes the similarity between two movies.**

The dot product between each vector will directly give you the cosine similarity score.
cosine_similarity(x,y) = (X-Transpose * Y) / (||X|| * ||Y||)
Please check the [link](https://www.machinelearningplus.com/nlp/cosine-similarity/#2whatiscosinesimilarityandwhyisitadvantageous) for understanding cosine similarities.

In my case the dataset is huge and the google colab session gets crashed. So I'm taking a fraction of the metadata dataframe and working on it. I will have to again go back and start from creating the TF-IDF matrix. You can try for the entire dataframe.

---



####Note : You can also use sklearn's linear_kernel() instead of cosine_similarities() since it is faster.

In [9]:
fraction_metadata=metadata.head(10000)

fraction_tfidf_matrix = tfidf.fit_transform(fraction_metadata['overview'])

fraction_tfidf_matrix.shape
# So here we get 32350 total unique words from Top 10000 overview entries of the original dataframe

(10000, 32350)

####**Back to computing the cosine similarities**

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim=cosine_similarity(fraction_tfidf_matrix, fraction_tfidf_matrix)

print(cosine_sim)

[[1.         0.01682915 0.         ... 0.         0.         0.        ]
 [0.01682915 1.         0.04871976 ... 0.         0.01200997 0.        ]
 [0.         0.04871976 1.         ... 0.         0.00735515 0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.08838493]
 [0.         0.01200997 0.00735515 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.08838493 0.         1.        ]]


In [11]:
cosine_sim.shape

(10000, 10000)

In [12]:
cosine_sim[5]

array([0.        , 0.05256713, 0.        , ..., 0.        , 0.03229893,
       0.        ])

**You're going to define a function that takes in a movie title as an input and outputs a list of the 10 most similar movies.** 
First you need a reverse mapping of movie titles and DataFrame indices. In other words, you need to identify the index of a movie in your metadata DataFrame( for us it's fraction of metadata), given its title.

In [13]:
indices = pd.Series(fraction_metadata.index, index=fraction_metadata['title']).drop_duplicates()

In [14]:
indices[:20]

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
Heat                               5
Sabrina                            6
Tom and Huck                       7
Sudden Death                       8
GoldenEye                          9
The American President            10
Dracula: Dead and Loving It       11
Balto                             12
Nixon                             13
Cutthroat Island                  14
Casino                            15
Sense and Sensibility             16
Four Rooms                        17
Ace Ventura: When Nature Calls    18
Money Train                       19
dtype: int64

####**We get the final solution:** 
Follow these steps:


1.  Get the index of the movie given its title.

2.  Get the list of cosine similarity scores for that particular movie with all movies. Convert it into a list of tuples where the first element is its position, and the second is the similarity score.

3.  Sort the list based on the similarity scores.

4.  Get the top 10 elements of this list ignoring the first element as it refers to self (the movie most similar to a particular movie is the movie itself).

5.  Return the movie titles corresponding to the indices of the top elements.


In [15]:
def recommend_movies(title, cosine_sim=cosine_sim):
    
    idx = indices[title]

    
    sim_scores = list(enumerate(cosine_sim[idx]))

    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    
    sim_scores = sim_scores[1:11]

    
    movie_indices = [i[0] for i in sim_scores]

    
    return fraction_metadata['title'].iloc[movie_indices]

In [16]:
recommend_movies('Toy Story')

2997                                    Toy Story 2
8327                                      The Champ
1071                          Rebel Without a Cause
3057                                Man on the Moon
1932                                      Condorman
485                                          Malice
5797                                  Class of 1984
7254                                 Africa Screams
6944                               Rivers and Tides
7615    The First $20 Million Is Always the Hardest
Name: title, dtype: object